In [12]:
# ============================================================
# Basic CNN on CIFAR-10 (PyTorch + CUDA)
# ============================================================

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F
import time

# ------------------------------------------------------------
# 1. Check device (GPU/CPU)
# ------------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ------------------------------------------------------------
# 2. Data transforms and loaders
# ------------------------------------------------------------
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), 
                         (0.2470, 0.2435, 0.2616))
])

train_dataset = datasets.CIFAR10(root="./data", train=True, 
                                 transform=transform, download=True)
test_dataset = datasets.CIFAR10(root="./data", train=False, 
                                transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)

# ------------------------------------------------------------
# 3. Define CNN model
# ------------------------------------------------------------
class BasicCNN(nn.Module):
    def __init__(self):
        super(BasicCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = BasicCNN().to(device)
print(model)

# ------------------------------------------------------------
# 4. Define loss and optimizer
# ------------------------------------------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ------------------------------------------------------------
# 5. Training loop
# ------------------------------------------------------------
epochs = 10
start_time = time.time()

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}] | Loss: {running_loss/len(train_loader):.4f} | Train Acc: {train_acc:.2f}%")

# ------------------------------------------------------------
# 6. Evaluation on test data
# ------------------------------------------------------------
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

test_acc = 100 * correct / total
print(f"\n✅ Test Accuracy: {test_acc:.2f}%")
print(f"⏱ Total Training Time: {(time.time() - start_time)/60:.2f} mins")


Using device: cuda
Files already downloaded and verified
Files already downloaded and verified
BasicCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=16384, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)
Epoch [1/10] | Loss: 1.6718 | Train Acc: 38.61%
Epoch [2/10] | Loss: 1.3640 | Train Acc: 50.54%
Epoch [3/10] | Loss: 1.2535 | Train Acc: 55.00%
Epoch [4/10] | Loss: 1.1914 | Train Acc: 57.68%
Epoch [5/10] | Loss: 1.1518 | Train Acc: 59.16%
Epoch [6/10] | Loss: 1.1341 | Train Acc: 59.80%
Epoch [7/10] | Loss: 1.1061 | Train Acc: 61.00%
Epoch [8/10] | Loss: 1.0873 | Train Acc: 61.90%
Epoch [9/10] | Loss: 1.0704 | Train Acc: 62.27%
Epoch [10/10] | Loss: 1.0539 | Train Acc: 62.86%

✅ Tes

In [ ]:
#One More Convolutional Layer
class CNN_MoreConv(nn.Module):
    def __init__(self):
        super(CNN_MoreConv, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(128 * 8 * 8, 256)  # one extra pool halves the size
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x
model = CNN_MoreConv().to(device)
print(model)

# ------------------------------------------------------------
# 4. Define loss and optimizer
# ------------------------------------------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ------------------------------------------------------------
# 5. Training loop
# ------------------------------------------------------------
epochs = 10
start_time = time.time()

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}] | Loss: {running_loss/len(train_loader):.4f} | Train Acc: {train_acc:.2f}%")

# ------------------------------------------------------------
# 6. Evaluation on test data
# ------------------------------------------------------------
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

test_acc = 100 * correct / total
print(f"\n✅ Test Accuracy: {test_acc:.2f}%")
print(f"⏱ Total Training Time: {(time.time() - start_time)/60:.2f} mins")



CNN_MoreConv(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=8192, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)
Epoch [1/10] | Loss: 1.6441 | Train Acc: 39.50%


In [ ]:
class CNN_LessConv(nn.Module):
    def __init__(self):
        super(CNN_LessConv, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(32 * 16 * 16, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x
    
model = CNN_LessConv().to(device)
print(model)

# ------------------------------------------------------------
# 4. Define loss and optimizer
# ------------------------------------------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ------------------------------------------------------------
# 5. Training loop
# ------------------------------------------------------------
epochs = 10
start_time = time.time()

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}] | Loss: {running_loss/len(train_loader):.4f} | Train Acc: {train_acc:.2f}%")

# ------------------------------------------------------------
# 6. Evaluation on test data
# ------------------------------------------------------------
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

test_acc = 100 * correct / total
print(f"\n✅ Test Accuracy: {test_acc:.2f}%")
print(f"⏱ Total Training Time: {(time.time() - start_time)/60:.2f} mins")


CNN_LessConv(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=8192, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)
Epoch [1/10] | Loss: 1.7926 | Train Acc: 34.00%
Epoch [2/10] | Loss: 1.6005 | Train Acc: 41.48%
Epoch [3/10] | Loss: 1.5353 | Train Acc: 43.79%
Epoch [4/10] | Loss: 1.4980 | Train Acc: 45.68%
Epoch [5/10] | Loss: 1.4685 | Train Acc: 46.88%
Epoch [6/10] | Loss: 1.4400 | Train Acc: 47.78%
Epoch [7/10] | Loss: 1.4198 | Train Acc: 48.72%
Epoch [8/10] | Loss: 1.4059 | Train Acc: 49.56%
Epoch [9/10] | Loss: 1.3871 | Train Acc: 49.89%
Epoch [10/10] | Loss: 1.3811 | Train Acc: 50.45%

✅ Test Accuracy: 58.64%
⏱ Total Training Time: 2.11 mins


In [ ]:
class CNN_BiggerKernel(nn.Module):
    def __init__(self):
        super(CNN_BiggerKernel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(64 * 16 * 16, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x
model = CNN_BiggerKernel().to(device)
print(model)

# ------------------------------------------------------------
# 4. Define loss and optimizer
# ------------------------------------------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ------------------------------------------------------------
# 5. Training loop
# ------------------------------------------------------------
epochs = 10
start_time = time.time()

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}] | Loss: {running_loss/len(train_loader):.4f} | Train Acc: {train_acc:.2f}%")

# ------------------------------------------------------------
# 6. Evaluation on test data
# ------------------------------------------------------------
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

test_acc = 100 * correct / total
print(f"\n✅ Test Accuracy: {test_acc:.2f}%")
print(f"⏱ Total Training Time: {(time.time() - start_time)/60:.2f} mins")


CNN_BiggerKernel(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=16384, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)
Epoch [1/10] | Loss: 1.6806 | Train Acc: 38.46%
Epoch [2/10] | Loss: 1.4098 | Train Acc: 48.94%
Epoch [3/10] | Loss: 1.3084 | Train Acc: 53.20%
Epoch [4/10] | Loss: 1.2491 | Train Acc: 55.18%
Epoch [5/10] | Loss: 1.2076 | Train Acc: 56.90%
Epoch [6/10] | Loss: 1.1726 | Train Acc: 58.24%
Epoch [7/10] | Loss: 1.1503 | Train Acc: 59.23%
Epoch [8/10] | Loss: 1.1349 | Train Acc: 59.99%
Epoch [9/10] | Loss: 1.1213 | Train Acc: 60.36%
Epoch [10/10] | Loss: 1.0986 | Train Acc: 61.18%

✅ Test Accuracy: 65.68%
⏱ Total Training Time: 2.84 mins


In [ ]:
class CNN_NoDropout(nn.Module):
    def __init__(self):
        super(CNN_NoDropout, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 16 * 16, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
model = CNN_NoDropout().to(device)
print(model)

# ------------------------------------------------------------
# 4. Define loss and optimizer
# ------------------------------------------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ------------------------------------------------------------
# 5. Training loop
# ------------------------------------------------------------
epochs = 10
start_time = time.time()

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}] | Loss: {running_loss/len(train_loader):.4f} | Train Acc: {train_acc:.2f}%")

# ------------------------------------------------------------
# 6. Evaluation on test data
# ------------------------------------------------------------
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

test_acc = 100 * correct / total
print(f"\n✅ Test Accuracy: {test_acc:.2f}%")
print(f"⏱ Total Training Time: {(time.time() - start_time)/60:.2f} mins")


CNN_NoDropout(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=16384, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)
Epoch [1/10] | Loss: 1.5624 | Train Acc: 43.47%
Epoch [2/10] | Loss: 1.2031 | Train Acc: 56.79%
Epoch [3/10] | Loss: 1.0635 | Train Acc: 61.99%
Epoch [4/10] | Loss: 0.9812 | Train Acc: 65.16%
Epoch [5/10] | Loss: 0.9246 | Train Acc: 67.09%
Epoch [6/10] | Loss: 0.8818 | Train Acc: 68.64%
Epoch [7/10] | Loss: 0.8460 | Train Acc: 70.18%
Epoch [8/10] | Loss: 0.8147 | Train Acc: 71.36%
Epoch [9/10] | Loss: 0.7858 | Train Acc: 72.46%
Epoch [10/10] | Loss: 0.7644 | Train Acc: 73.25%

✅ Test Accuracy: 71.74%
⏱ Total Training Time: 1.99 mins


In [ ]:
class CNN_AvgPool(nn.Module):
    def __init__(self):
        super(CNN_AvgPool, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.AvgPool2d(2, 2)  # changed pooling
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(64 * 16 * 16, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x
model = CNN_AvgPool().to(device)
print(model)

# ------------------------------------------------------------
# 4. Define loss and optimizer
# ------------------------------------------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ------------------------------------------------------------
# 5. Training loop
# ------------------------------------------------------------
epochs = 10
start_time = time.time()

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}] | Loss: {running_loss/len(train_loader):.4f} | Train Acc: {train_acc:.2f}%")

# ------------------------------------------------------------
# 6. Evaluation on test data
# ------------------------------------------------------------
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

test_acc = 100 * correct / total
print(f"\n✅ Test Accuracy: {test_acc:.2f}%")
print(f"⏱ Total Training Time: {(time.time() - start_time)/60:.2f} mins")


CNN_AvgPool(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=16384, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)
Epoch [1/10] | Loss: 1.6589 | Train Acc: 39.45%
Epoch [2/10] | Loss: 1.3749 | Train Acc: 49.96%
Epoch [3/10] | Loss: 1.2626 | Train Acc: 54.79%
Epoch [4/10] | Loss: 1.2012 | Train Acc: 57.18%
Epoch [5/10] | Loss: 1.1540 | Train Acc: 59.05%
Epoch [6/10] | Loss: 1.1173 | Train Acc: 60.31%
Epoch [7/10] | Loss: 1.0952 | Train Acc: 61.15%
Epoch [8/10] | Loss: 1.0701 | Train Acc: 62.31%
Epoch [9/10] | Loss: 1.0495 | Train Acc: 62.80%
Epoch [10/10] | Loss: 1.0298 | Train Acc: 63.64%

✅ Test Accuracy: 68.80%
⏱ Total Training Time: 1.96 mins


In [ ]:
class CNN_MorePooling(nn.Module):
    def __init__(self):
        super(CNN_MorePooling, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(64 * 8 * 8, 256)  # smaller spatial size after extra pooling
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))   # first pooling
        x = self.pool(F.relu(self.conv2(x)))   # second pooling
        x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x
model = CNN_MorePooling().to(device)
print(model)

# ------------------------------------------------------------
# 4. Define loss and optimizer
# ------------------------------------------------------------
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ------------------------------------------------------------
# 5. Training loop
# ------------------------------------------------------------
epochs = 10
start_time = time.time()

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{epochs}] | Loss: {running_loss/len(train_loader):.4f} | Train Acc: {train_acc:.2f}%")

# ------------------------------------------------------------
# 6. Evaluation on test data
# ------------------------------------------------------------
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

test_acc = 100 * correct / total
print(f"\n✅ Test Accuracy: {test_acc:.2f}%")
print(f"⏱ Total Training Time: {(time.time() - start_time)/60:.2f} mins")


CNN_MorePooling(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=4096, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)
Epoch [1/10] | Loss: 1.6821 | Train Acc: 38.12%
Epoch [2/10] | Loss: 1.4183 | Train Acc: 48.30%
Epoch [3/10] | Loss: 1.3049 | Train Acc: 52.86%
Epoch [4/10] | Loss: 1.2253 | Train Acc: 56.05%
Epoch [5/10] | Loss: 1.1732 | Train Acc: 58.17%
Epoch [6/10] | Loss: 1.1359 | Train Acc: 59.58%
Epoch [7/10] | Loss: 1.1083 | Train Acc: 60.83%
Epoch [8/10] | Loss: 1.0833 | Train Acc: 61.76%
Epoch [9/10] | Loss: 1.0586 | Train Acc: 62.67%
Epoch [10/10] | Loss: 1.0433 | Train Acc: 63.25%

✅ Test Accuracy: 68.27%
⏱ Total Training Time: 1.64 mins
